# Ontologie des pokémon

In [1]:
from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
from unidecode import unidecode
import json

# Charger les données JSON
with open('./data/pokemons.json', 'r', encoding='utf-8') as file:
    pokemons_data = json.load(file)

# Créer une nouvelle ontologie
onto = get_ontology("http://example.org/onto/pokemon")

with onto:
    
    # Les classes caractérisant les pokémons
    class Pokemon(Thing):
        pass

    class Type(Thing):
        pass
    
    class Stat(Thing):
        pass
    
    class Sensi(Thing):
        pass

    class Attack(Thing):
        pass

    # Type du pokémon
    class hasType(ObjectProperty):
        domain = [Pokemon]
        range = [Type]
        
    # Statistiques du pokémon
    class hasStat(ObjectProperty):
        domain = [Pokemon]
        range = [Stat]
        
    # Sensibilités du pokémon
    class hasSensi(ObjectProperty):
        domain = [Pokemon]
        range = [Sensi]
        
    # Attaques du pokémon
    class hasAttack(ObjectProperty):
        domain = [Pokemon]
        range = [Attack]
        
    # Nom du pokémon
    class nom(DataProperty, FunctionalProperty):
         domain = [Pokemon, Type, Stat, Sensi, Attack]
         range = [str]

    # Valeur de la statistique
    class valeur(DataProperty, FunctionalProperty): 
        domain = [Stat, Sensi]
        range = [float]
    
    # Type de sensibilité du pokémon
    class hasSensiType(ObjectProperty, FunctionalProperty):
        domain = [Sensi]
        range = [Type]
    
    # Type d'attaque du pokémon
    class hasAttackType(ObjectProperty):
        domain = [Attack]
        range = [Type]
    
    # Categorie de l'attaque
    class categorie(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [str]

    #  Puissance de l'attaque du pokémon
    class puissance(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    #  Precision de l'attaque du pokémon
    class precision(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    #  PP de l'attaque du pokémon
    class pp(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    # Pré-remplir les types pour les réutilisation
    types = {name: Type(nom=name) for name in ["Normal", "Plante", "Feu", "Eau", "Electrik", "Glace", "Combat", "Poison", "Sol", "Vol", "Psy", "Insecte", "Roche", "Spectre", "Dragon", "Tenebres", "Acier", "Fee"]}
    types["Inconnu"] = Type(nom="Inconnu")  # Gérer les cas où le type est absent

    # Dictionnaire permettant de servir de cache pour éviter les multiples instances
    stats_cache = {}
    sensi_cache = {}
    attacks = {}

    # Gérer la création des Pokémons et leurs caractéristiques
    for pokemon_data in pokemons_data:
        
        # Ajouter le nom de chaque pokemon
        pokemon_nom = unidecode(pokemon_data['nom']) 
        pokemon_instance = Pokemon(nom=pokemon_nom)

        # Attribuer les types au pokémon
        for type_name in pokemon_data['types']:
            pokemon_instance.hasType.append(types[unidecode(type_name)])

        # Gérer les statistiques du pokémon
        for stat_name, stat_value in pokemon_data['stats'].items():
            # Clé pour le cache basée sur le nom et la valeur
            key = (unidecode(stat_name), float(stat_value))
            if key not in stats_cache:
                stats_cache[key] = Stat(nom=key[0], valeur=key[1])
            pokemon_instance.hasStat.append(stats_cache[key])
            
        # Gérer les sensibilités du pokémon
        for sensi_name, sensi_value in pokemon_data['sensibilities'].items():
            # Clé pour le cache basée uniquement sur le type (sans nom)
            type_ref = types[unidecode(sensi_name)]
            key = (type_ref, float(sensi_value))  
            if key not in sensi_cache:
                sensi_instance = Sensi(valeur=key[1])
                sensi_instance.hasSensiType = type_ref  
                sensi_cache[key] = sensi_instance
            pokemon_instance.hasSensi.append(sensi_cache[key])

        # Attaques du Pokémon
        for attack_data in pokemon_data['attaques']:
            attack_key = unidecode(attack_data['Nom'])
            if attack_key not in attacks:
                attack_type_name = attack_data['Type']
                if attack_type_name:
                    attack_type_name = unidecode(attack_type_name)
                    if attack_type_name not in types:
                        attack_type_instance = Type(nom=attack_type_name)
                        types[attack_type_name] = attack_type_instance
                    else:
                        attack_type_instance = types[attack_type_name]
                    attack_instance = Attack(
                        nom=attack_key,
                        categorie=unidecode(attack_data['Catégorie']) if attack_data['Catégorie'] else "Inconnu",
                        puissance=attack_data['Puissance'] if attack_data['Puissance'] is not None else 0,
                        precision=attack_data['Précision'] if attack_data['Précision'] is not None else 0,
                        pp=attack_data['PP'] if attack_data['PP'] is not None else 0
                    )
                    attack_instance.hasAttackType.append(attack_type_instance)
                else:
                    # Attaque inconnu en cas d'erreur
                    attack_instance = Attack(
                        nom=attack_key,
                        categorie="Inconnu",
                        puissance=0,
                        precision=0,
                        pp=0
                    )
                attacks[attack_key] = attack_instance
            pokemon_instance.hasAttack.append(attacks[attack_key])

    # Enregistrer l'ontologie
            
    onto.save(file="pokemon_ontology.owl", format="rdfxml")
    print("Ontology créé et sauvegardée.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontology créé et sauvegardée.


In [2]:
import json
from owlready2 import get_ontology, sync_reasoner, default_world

# Charger l'ontologie à partir du fichier
onto_path = "./pokemon_ontology.owl" 
onto = get_ontology(onto_path).load()

# Moteur de raisonnement 
sync_reasoner()


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/basti/AppData/Local/Temp/tmpuqxqfrxt
* Owlready2 * HermiT took 3.713459014892578 seconds
* Owlready * Reparenting pokemon.attack395: {pokemon.Attack} => {pokemon.Pokemon, pokemon.Stat, pokemon.Attack, pokemon.Sensi, pokemon.Type}
* Owlready * Reparenting pokemon.stat311: {pokemon.Stat} => {pokemon.Pokemon, pokemon.Stat, pokemon.Attack, pokemon.Sensi, pokemon.Type}
* Owlready * Reparenting pokemon.pokemon314: {pokemon.Pokemon} => {pokemon.Pokemon, pokemon.Stat, pokemon.Attack, pokemon.Sensi, pokemon.Type}
* Owlready * Reparenting pokemon.attack396: {pokemon.Attack} => {pokemon.Pokemon, pokemon.Stat, pokemon.Attack, pokemon.Sensi, pokemon.Type}
* Owlready * Reparenting pokemon.st

# Requête sur les pokémons

Requête pour lister tout les pokémons de type feu

In [3]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>
SELECT DISTINCT ?pokemon WHERE {
    ?pokemon rdf:type onto:Pokemon .
    ?pokemon onto:hasType ?type .
    ?type onto:nom "Feu" .
}
"""
# Exécution de la requête
results = list(default_world.sparql(query))

# Affichage des résultats
for result in results:
    print(f"Pokémon de type Feu : {result[0].nom}")


Pokémon de type Feu : Salameche
Pokémon de type Feu : Reptincel
Pokémon de type Feu : Dracaufeu
Pokémon de type Feu : Goupix
Pokémon de type Feu : Feunard
Pokémon de type Feu : Caninos
Pokémon de type Feu : Arcanin
Pokémon de type Feu : Ponyta
Pokémon de type Feu : Galopa
Pokémon de type Feu : Magmar
Pokémon de type Feu : Pyroli
Pokémon de type Feu : Sulfura
Pokémon de type Feu : Ouisticram
Pokémon de type Feu : Chimpenfeu
Pokémon de type Feu : Simiabraz
Pokémon de type Feu : Maganon
Pokémon de type Feu : Heatran
Pokémon de type Feu : Hericendre
Pokémon de type Feu : Feurisson
Pokémon de type Feu : Typhlosion
Pokémon de type Feu : Limagma
Pokémon de type Feu : Volcaropod
Pokémon de type Feu : Malosse
Pokémon de type Feu : Demolosse
Pokémon de type Feu : Magby
Pokémon de type Feu : Entei
Pokémon de type Feu : Ho-Oh
Pokémon de type Feu : Poussifeu
Pokémon de type Feu : Galifeu
Pokémon de type Feu : Brasegali
Pokémon de type Feu : Chamallot
Pokémon de type Feu : Camerupt
Pokémon de type F

Une requête pour montrer les pokémons sensibles aux types d'un pokémon cible

In [4]:
def afficher_pokemons_contre(reference_pokemon):
    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?otherPokemon WHERE {{
        ?pokemon onto:nom "{reference_pokemon}" .
        ?pokemon onto:hasType ?pokemonType .
        ?sensi onto:hasSensiType ?pokemonType .
        ?sensi onto:valeur ?value .
        FILTER(?value >= 4) .
        ?otherPokemon onto:hasSensi ?sensi .
    }}
    """


    # Exécution de la requête
    results = list(default_world.sparql(query))

    # Affichage des résultats
    for result in results:
        pokemon = result[0]
        print(f"Pokémon possédant une double sensibilitées au type de Dracaufeu : {pokemon.nom}")

afficher_pokemons_contre("Dracaufeu")

Pokémon possédant une double sensibilitées au type de Dracaufeu : Paras
Pokémon possédant une double sensibilitées au type de Dracaufeu : Parasect
Pokémon possédant une double sensibilitées au type de Dracaufeu : Cheniselle
Pokémon possédant une double sensibilitées au type de Dracaufeu : Blizzi
Pokémon possédant une double sensibilitées au type de Dracaufeu : Blizzaroi
Pokémon possédant une double sensibilitées au type de Dracaufeu : Foretress
Pokémon possédant une double sensibilitées au type de Dracaufeu : Cizayox
Pokémon possédant une double sensibilitées au type de Dracaufeu : Scarhino
Pokémon possédant une double sensibilitées au type de Dracaufeu : Chapignon


Voici une requête qui permet de lister tous les pokémons qui possède une sensibilitée = 4 aux attaques d'un pokémon particulier

In [5]:

def afficher_sensibilites_pokemon(reference_pokemon):
    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?typeAttaqueNom ?valeurSensi
    WHERE {{
        ?pokemonAttaquant onto:nom "{reference_pokemon}" .
        ?pokemonAttaquant onto:hasAttack ?attack .
        ?attack onto:hasAttackType ?typeAttaque .
        ?typeAttaque onto:nom ?typeAttaqueNom .
        
        ?pokemonCible onto:hasSensi ?sensi .
        ?sensi onto:hasSensiType ?typeAttaque .
        ?sensi onto:valeur ?valeurSensi .
        FILTER(?valeurSensi = 4) .
        ?pokemonCible onto:nom ?pokemonNom .
    }}
    ORDER BY ?pokemonNom
    """

    results = default_world.sparql(query)
    print(f"Pokémons sensibles aux types d'attaques de {reference_pokemon} :")
    for result in results:
        pokemon_nom, type_attaque_nom, valeur_sensi = result
        print(f"- {pokemon_nom} est sensible au type {type_attaque_nom} avec une sensibilité de {valeur_sensi}.")
        
# Utilisation de la fonction pour "Dracaufeu"
afficher_sensibilites_pokemon("Dracaufeu")


Pokémons sensibles aux types d'attaques de Dracaufeu :
- Apireine est sensible au type Roche avec une sensibilité de 4.0.
- Apitrini est sensible au type Roche avec une sensibilité de 4.0.
- Artikodin est sensible au type Roche avec une sensibilité de 4.0.
- Bastiodon est sensible au type Combat avec une sensibilité de 4.0.
- Bastiodon est sensible au type Sol avec une sensibilité de 4.0.
- Blizzaroi est sensible au type Feu avec une sensibilité de 4.0.
- Blizzi est sensible au type Feu avec une sensibilité de 4.0.
- Cadoizo est sensible au type Roche avec une sensibilité de 4.0.
- Chapignon est sensible au type Vol avec une sensibilité de 4.0.
- Charmillon est sensible au type Roche avec une sensibilité de 4.0.
- Cheniselle est sensible au type Feu avec une sensibilité de 4.0.
- Cizayox est sensible au type Feu avec une sensibilité de 4.0.
- Coxy est sensible au type Roche avec une sensibilité de 4.0.
- Coxyclaque est sensible au type Roche avec une sensibilité de 4.0.
- Dimoret est s

Afficher la puissance et les types des attaques d'un pokémon

In [6]:

# Requête SPARQL
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX onto: <http://example.org/onto/pokemon#>

SELECT DISTINCT ?attackName ?attackType ?power
WHERE {
  ?pokemon onto:nom "Dracaufeu" .
  ?pokemon onto:hasAttack ?attack .
  ?attack onto:nom ?attackName .
  ?attack onto:puissance ?power .
  ?attack onto:hasAttackType ?type .
  ?type onto:nom ?attackType
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Nom de l'attaque: {result[0]}, Type: {result[1]}, Puissance: {result[2]}")


Nom de l'attaque: Lame d'Air, Type: Vol, Puissance: 75
Nom de l'attaque: Canicule, Type: Feu, Puissance: 95
Nom de l'attaque: Draco-Griffe, Type: Dragon, Puissance: 80
Nom de l'attaque: Griffe, Type: Normal, Puissance: 40
Nom de l'attaque: Rugissement, Type: Normal, Puissance: 0
Nom de l'attaque: Flammeche, Type: Feu, Puissance: 40
Nom de l'attaque: Brouillard, Type: Normal, Puissance: 0
Nom de l'attaque: Draco-Souffle, Type: Dragon, Puissance: 60
Nom de l'attaque: Crocs Feu, Type: Feu, Puissance: 65
Nom de l'attaque: Tranche, Type: Normal, Puissance: 70
Nom de l'attaque: Lance-Flammes, Type: Feu, Puissance: 90
Nom de l'attaque: Grimace, Type: Normal, Puissance: 0
Nom de l'attaque: Danse Flammes, Type: Feu, Puissance: 35
Nom de l'attaque: Feu d'Enfer, Type: Feu, Puissance: 100
Nom de l'attaque: Boutefeu, Type: Feu, Puissance: 120
Nom de l'attaque: Pouvoir Antique, Type: Roche, Puissance: 60
Nom de l'attaque: Cognobidon, Type: Normal, Puissance: 0
Nom de l'attaque: Morsure, Type: Tenebr

Afficher les sensibilités d'un pokémon en particulier

In [7]:
from owlready2 import default_world

# Assurez-vous que votre ontologie est chargée
onto = get_ontology("http://example.org/onto/pokemon").load()

# La requête SPARQL
query = """
PREFIX : <http://example.org/onto/pokemon#>
SELECT DISTINCT ?typeName ?sensiValue
WHERE {
  ?pokemon :nom "Dracaufeu" .
  ?pokemon :hasSensi ?sensi .
  ?sensi :hasSensiType ?type .
  ?type :nom ?typeName .
  ?sensi :valeur ?sensiValue .
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Type: {result[0]}, Sensibilité (Valeur): {result[1]}")


Type: Normal, Sensibilité (Valeur): 1.0
Type: Plante, Sensibilité (Valeur): 0.25
Type: Feu, Sensibilité (Valeur): 0.5
Type: Eau, Sensibilité (Valeur): 2.0
Type: Electrik, Sensibilité (Valeur): 2.0
Type: Glace, Sensibilité (Valeur): 1.0
Type: Combat, Sensibilité (Valeur): 0.5
Type: Poison, Sensibilité (Valeur): 1.0
Type: Sol, Sensibilité (Valeur): 0.0
Type: Vol, Sensibilité (Valeur): 1.0
Type: Psy, Sensibilité (Valeur): 1.0
Type: Insecte, Sensibilité (Valeur): 0.25
Type: Roche, Sensibilité (Valeur): 4.0
Type: Spectre, Sensibilité (Valeur): 1.0
Type: Dragon, Sensibilité (Valeur): 1.0
Type: Tenebres, Sensibilité (Valeur): 1.0
Type: Acier, Sensibilité (Valeur): 0.5
Type: Fee, Sensibilité (Valeur): 0.5


Afficher les statistiques d'un pokémon en particulier

In [8]:
from owlready2 import default_world

# Assurez-vous que votre ontologie est chargée
onto = get_ontology("http://example.org/onto/pokemon").load()

# La requête SPARQL
query = """
PREFIX : <http://example.org/onto/pokemon#>
SELECT DISTINCT ?statName ?value
WHERE {
  ?pokemon :nom "Dracaufeu" .
  ?pokemon :hasStat ?stat .
  ?stat :nom ?statName .
  ?stat :valeur ?value .
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Statistique: {result[0]}, Valeur: {result[1]}")


Statistique: PV, Valeur: 78.0
Statistique: Attaque, Valeur: 84.0
Statistique: Defense, Valeur: 78.0
Statistique: Attaque Speciale, Valeur: 109.0
Statistique: Defense Speciale, Valeur: 85.0
Statistique: Vitesse, Valeur: 100.0
Statistique: Special, Valeur: 85.0


# Réponse à la problématique initiale avec les attaques 

In [9]:
# Liste des Pokémon du joueur
pokemons_joueur = ["Dracaufeu", "Dardargnan", "Dracolosse", "Raichu", "Entei", "Nosferalto"]

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power ?sensiValue
    WHERE {{
        # On récupère les noms et les attaques d'un pokémon afin de connaitre leurs types et leurs puissance et connaitre le type du pokémon
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?typeAttaque .
        ?typeAttaque onto:nom ?attackType .
        ?attack onto:puissance ?power .
        
        # On récupère les sensibilités du pokémons adverse et ses types afin de privilés les attaques efficace
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasSensi ?sensiAdv .
        ?sensiAdv onto:hasSensiType ?typeAttaque .
        ?sensiAdv onto:valeur ?sensiValue .
        FILTER (?sensiValue >= 2) .
        
        # Vérifier que le Pokémon n'est pas vulnérable à aucun type de l'adversaire
        FILTER NOT EXISTS {{
            ?adversaire onto:hasType ?advType .
            ?pokemon onto:hasSensi ?pokemonSensi .
            ?pokemonSensi onto:hasSensiType ?advType .
            ?pokemonSensi onto:valeur ?sensiVuln .
            FILTER (?sensiVuln > 1) .
        }}
    }}
    
    # On tri nos résultats par ordre de puissance des attaques pour montrer les meilleures attaques
    ORDER BY DESC(?sensiValue) DESC(?power)
    
    # On affiche les 3 meilleurs résultats obtenu
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        pokemon_nom = result[0] if result[0] else "Inconnu"
        attack_name = result[1] if result[1] else "Inconnue"
        attack_type = result[2] if result[2] else "Type Inconnu"
        power = result[3] if result[3] else "Puissance Inconnue"
        sensiValue = result[4] if result[4] else "Efficacité Inconnue"
        print(f" - {pokemon_nom} avec l'attaque '{attack_name}' (Type: {attack_type}) ayant une puissance de {power} et une efficacité de {sensiValue}.")

# Exécution de la fonction pour chaque adversaire
for adversaire in ["Lamantine", "Florizarre", "Hyporoi", "Pyroli", "Lippoutou", "Amonistar"]:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)


Contre Lamantine, les meilleures attaques à utiliser sont :


 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.0.
Contre Florizarre, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150 et une efficacité de 2.0.
 - Nosferalto avec l'attaque 'Rapace' (Type: Vol) ayant une puissance de 120 et une efficacité de 2.0.
 - Dracaufeu avec l'attaque 'Boutefeu' (Type: Feu) ayant une puissance de 120 et une efficacité de 2.0.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Calinerie' (Type: Fee) ayant une puissance de 90 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Voix Enjoleuse' (Type: Fee) ayant une puissance de 40 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Charme' (Type